# Teradata as Embeddings Storage. Semantic search

## Introduction

Embeddings are revolutionizing the way we process and understand text data. These dense vector representations capture the semantic essence of words, phrases, and even entire documents, enabling machines to grasp nuanced meanings and relationships within the data. Imagine a high-dimensional space where each point represents a word, phrase, or document as a vector. The position of each point is determined by the context in which it appears, meaning similar concepts are located closer together.

For instance, the words "king" and "queen" would have vectors that are close to each other, as would "Paris" and "France." This spatial proximity allows embeddings to capture intricate relationships and patterns that traditional keyword-based methods miss.

Unlike traditional keyword-based search methods, embedding-based search leverages these rich representations to find relevant information based on context and meaning rather than exact word matches. This is where cosine similarity comes into play. By measuring the cosine of the angle between vectors, cosine similarity allows us to quantify how similar two embeddings are, effectively identifying the most relevant documents or texts. This technique is crucial in applications ranging from information retrieval and recommendation systems to natural language understanding, providing more accurate and meaningful search results.

### Illustration of Embeddings

To visualize embeddings, imagine a 2D plot (though embeddings typically exist in much higher dimensions):

![alt text](img/embeddings_1.jpg "Embeddings: King and Queen, Paris and France")


In this illustration:
- "King" and "Queen" are close together, indicating they are semantically similar.
- "Paris" and "France" are also close together, showing a geographical relationship.

By using embeddings, we can better understand and search through our data in ways that are meaningful and contextually relevant.

## Approach

In this demo, we showcase an advanced approach to embedding-based search using the Teradata database. Our methodology involves several key steps:

1. **Importing and Converting Model**: We begin by importing pre-trained models from Hugging Face, which are renowned for their ability to capture semantic meanings in text data effectively. To enhance performance and ensure compatibility with various execution environments, we convert these Hugging Face models into the ONNX (Open Neural Network Exchange) format using the [`optimum`](https://github.com/huggingface/optimum) utility.

2. **Model Deployment to Database to be Used with BYOM**: Leveraging Teradata's BYOM (Bring Your Own Model) capability, we deploy the model directly within the Teradata database. This integration minimizes data movement and optimizes performance by keeping the model execution close to the data storage.

3. **In-Database Embedding Generation and Building the Embedding Store**: We execute the embedding generation process directly within the Teradata database. Each text entry in our knowledge base is processed to create its corresponding embedding vector, which is then stored in a structured repository for efficient retrieval.

4. **Semantic Search with Cosine Similarity**: Finally, we utilize Teradata’s functionality to calculate cosine similarity between a query embedding and the embeddings stored in the database. Cosine similarity, which measures the angle between two vectors, effectively determines their similarity. This enables us to perform semantic searches directly within the database, retrieving the most relevant results based on the meaning of the text rather than exact keyword matches.

The advantage of this approach is that the data never leaves the database. This ensures data security and compliance while reducing latency and improving efficiency, as all operations are performed close to where the data resides.

This approach combines state-of-the-art embedding models with Teradata's robust data management and processing capabilities, facilitating efficient and accurate semantic searches at scale.


![alt text](img/embeddings_diagram.jpg "Teradata in-database Embedding Store")

The advantage of this approach is that the data never leaves the database. This ensures data security and compliance while reducing latency and improving efficiency, as all operations are performed close to where the data resides.

This approach combines state-of-the-art embedding models with Teradata's robust data management and processing capabilities, facilitating efficient and accurate semantic searches at scale.



## Part 1. Importing and Converting Model

We start by importing the pre-trained [BAAI/bge](https://huggingface.co/BAAI/bge-small-en-v1.5) model from Hugging Face, renowned for its effectiveness in capturing semantic meanings in text data. The BAAI/bge model is a state-of-the-art model trained on a large corpus, capable of generating high-quality text embeddings.

To enhance performance and ensure compatibility with various execution environments, we'll use the Optimum utility to convert the model into the ONNX (Open Neural Network Exchange) format.

In [ ]:
! optimum-cli export onnx --opset 16 --trust-remote-code -m BAAI/bge-small-en-v1.5 bge-small-en-v1.5-onnx

After conversion to ONNX, we perform the following fixes:
- Fixing dynamic dimensions on input and output, ensuring compatibility with different input sizes.
- Fixing the opset in the ONNX file for compatibility with ONNX runtime.
- Removing tokens embeddings output to save I/O during processing, optimizing the model for efficient execution.

In [ ]:
import onnx
import onnxruntime as rt

import transformers
from onnxruntime.tools.onnx_model_utils import *

from sentence_transformers.util import cos_sim
from sentence_transformers import SentenceTransformer

import teradataml as tdml

import getpass

Once the fixes are applied, we proceed to test the correctness of the ONNX model by calculating cosine similarity between two texts using native SentenceTransformers and ONNX runtime, comparing the results.

If the results are identical, it confirms that the ONNX model gives the same result as the native models, validating its correctness and suitability for further use in the database.

In [ ]:
sentences_1 = 'How is the weather today?'
sentences_2 = 'What is the current weather like today?'

In [ ]:
# Calculate ONNX result

tokenizer = transformers.AutoTokenizer.from_pretrained("./bge-small-en-v1.5-onnx")
predef_sess = rt.InferenceSession("bge-small-en-v1.5-onnx/model.onnx")

enc1 = tokenizer(sentences_1, max_length = 512, padding='max_length' )
embeddings_1_onnx = predef_sess.run(None,     {"input_ids": [enc1.input_ids], 
     "attention_mask": [enc1.attention_mask]})

enc2 = tokenizer(sentences_2, max_length = 512, padding='max_length' )
embeddings_2_onnx = predef_sess.run(None,     {"input_ids": [enc2.input_ids], 
     "attention_mask": [enc2.attention_mask]})

In [ ]:
# Calculate native model result using SentenceTransformer

model = SentenceTransformer('BAAI/bge-small-en-v1.5')
embeddings_1_sentence_transformer = model.encode(sentences_1, normalize_embeddings=True)
embeddings_2_sentence_transformer = model.encode(sentences_2, normalize_embeddings=True)

In [ ]:
# Compare results

print("Cosine similiarity for embeddings calculated with ONNX:" + str(cos_sim(embeddings_1_onnx[1][0], embeddings_2_onnx[1][0])))
print("Cosine similiarity for embeddings calculated with SentenceTransformer:" + str(cos_sim(embeddings_1_sentence_transformer, embeddings_2_sentence_transformer)))

## Part 2. Model Deployment to Database to be Used with BYOM

In this section, we demonstrate how to deploy the model to the Teradata database using the BYOM (Bring Your Own Model) capability. We use the `teradataml` Python library to manage the connectivity and provide a convenient Python API that is similar to PySpark or pandas DataFrame.




### Opening Connection to Teradata

We start by setting up a connection to the Teradata database. The `teradataml` library handles all the intricacies of database connectivity, allowing us to interact with Teradata in a manner similar to working with data in pandas DataFrames.


In [ ]:
tdml.create_context(host = 'teradata', username='<YOUR USERNAME>', password = '<YOUR PASSWORD>')

### Deploying the Model and Tokenizer

After establishing the connection, we deploy two key artifacts to the database:
1. The model itself, converted to ONNX format.
2. The `tokenizer.json` file, which will be used for in-database tokenization.

Both artifacts are deployed using the `save_byom` function, which abstracts the underlying complexity and makes the deployment process straightforward. Internally, this function performs an insert operation into the database.

By using the `save_byom` function, we ensure that our model and tokenizer are readily available within the Teradata database for subsequent embedding generation and semantic search operations. This integration minimizes data movement and optimizes performance by keeping all operations within the database environment.

In [ ]:
#UNCOMMENT IF TABLE EXISTS
tdml.db_drop_table('embeddings_models')
tdml.save_byom('bge-small-en-v1.5',
              'bge-small-en-v1.5-onnx/model.onnx',
              'embeddings_models')

#UNCOMMENT IF TABLE EXISTS
tdml.db_drop_table('embeddings_tokenizers')
tdml.save_byom('bge-small-en-v1.5',
              'bge-small-en-v1.5-onnx/tokenizer.json',
              'embeddings_tokenizers')

## Part 3. In-Database Embedding Generation and Building the Embedding Store

In this point, we are taking the history of the emails and building the embedding store in one simple step:


In [ ]:
#UNCOMMENT IF TABLE EXISTS
#tdml.db_drop_table('emails_embeddings_store')

tdml.execute_sql("""

create table emails_embeddings_store as (
    select 
            *
    from mldb.ONNXEmbeddings(
            on emails.emails as InputTable
            on (select * from embeddings_models where model_id = 'bge-small-en-v1.5') as ModelTable DIMENSION
            on (select model as tokenizer from embeddings_tokenizers where model_id = 'bge-small-en-v1.5') as TokenizerTable DIMENSION
       
            using
                Accumulate('id', 'txt') 
                ModelOutputTensor('sentence_embedding')
                EnableMemoryCheck('false')
                OutputFormat('FLOAT32(384)')
        ) a 
) with data

""")

In [ ]:
tdf_embeddings_store = tdml.DataFrame('emails_embeddings_store')
tdf_embeddings_store.head(3)

By following these steps, we efficiently generate and store embeddings within the Teradata database, making them readily available for high-performance semantic search operations.


Building the embedding store directly within the Teradata database is both important and beneficial for several reasons:

- **Performance**: By generating and storing embeddings in-database, we reduce data movement and leverage Teradata’s powerful processing capabilities. This results in faster query execution and lower latency.

- **Scalability**: Teradata is designed to handle large-scale data. Embedding generation and storage within Teradata ensures that we can scale our operations to handle vast amounts of text data without compromising on performance.

- **Security**: Keeping data within the database ensures that sensitive information remains secure and complies with data governance policies. There is no need to move data to external systems for processing.

- **Integration**: Embedding the store directly in Teradata allows seamless integration with existing data and applications. This enables more comprehensive data analysis and supports advanced use cases such as real-time semantic search and analytics.

By leveraging Teradata's robust infrastructure and advanced capabilities, we can build an efficient, secure, and scalable embedding store that enhances our ability to perform sophisticated text analysis and semantic search.

## Part 4. Semantic Search with Cosine Similarity

In this final step, we perform semantic search using cosine similarity within the Teradata database. We utilize the `TD_VectorDistance` function, which is specifically designed for calculating cosine similarity between texts in our embedding store and given examples. This function leverages Teradata's Massive Parallel Processing (MPP) capabilities, enabling high-performance and scalable computation.

The `TD_VectorDistance` function computes the cosine similarity between the query embedding (representing the given example) and the embeddings stored in our embedding store. By comparing the angles between vectors in the multi-dimensional space, the function identifies the most semantically similar emails to the given example.

In this specific case, we aim to collect the most semantically similar emails by the given example. This allows us to efficiently identify relevant content and extract valuable insights from our email dataset.

By utilizing Teradata's powerful processing capabilities and in-database functions like `TD_VectorDistance`, we can perform advanced semantic search operations with unparalleled performance and scalability. This enables us to effectively analyze large volumes of text data and extract meaningful information, facilitating data-driven decision-making and enhancing business outcomes.


In [ ]:
tdf_embeddings_store_tgt = tdf_embeddings_store[tdf_embeddings_store.id == 3]
tdf_embeddings_store_ref = tdf_embeddings_store[tdf_embeddings_store.id != 3]

In [ ]:
tdml.DataFrame.from_query(f"""

SELECT 
    dt.target_id, 
    dt.reference_id,
    e_tgt.txt as target_txt,
    e_ref.txt as reference_txt,
    (1.0 - dt.distance) as similiarity 
FROM
    TD_VECTORDISTANCE (
        ON (%s) AS TargetTable
        ON (%s) AS ReferenceTable DIMENSION
        USING
            TargetIDColumn('id')
            TargetFeatureColumns('[emb_0:emb_383]')
            RefIDColumn('id')
            RefFeatureColumns('[emb_0:emb_383]')
            DistanceMeasure('cosine')
            topk(3)
    ) AS dt
JOIN emails.emails e_tgt on e_tgt.id = dt.target_id
JOIN emails.emails e_ref on e_ref.id = dt.reference_id;
"""%(tdf_embeddings_store_tgt.show_query(), tdf_embeddings_store_ref.show_query()))

In [ ]:
tdml.remove_context()